In [2]:
import numpy as np
import networkx as nx
import netgraph
import matplotlib.pylab as plt
import os
from tqdm import tqdm

os.makedirs("./Karate", exist_ok=True)

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.samplers.regular.leiden_sampler import LeidenSampler
from Qommunity.samplers.regular.louvain_sampler import LouvainSampler
from Qommunity.samplers.regular.dqm_sampler import DQMSampler
from Qommunity.samplers.regular.bayan_sampler import BayanSampler

from iterative_searcher.iterative_searcher import IterativeSearcher

In [3]:
K = nx.karate_club_graph()
num_runs = 50
resolution = 1

In [33]:
adv_sampler = AdvantageSampler(K, resolution=resolution, num_reads=100, use_clique_embedding=True)
adv_iterative= IterativeSearcher(adv_sampler)

saving_path = f"./Karate/solver-adv_runs-{num_runs}"
adv_communities, adv_modularities, adv_times_elapsed = adv_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

100%|███████████████████████████████████████████| 50/50 [07:03<00:00,  8.47s/it]


In [35]:
louv_sampler = LouvainSampler(K, resolution=resolution)
louv_iterative= IterativeSearcher(louv_sampler)

saving_path = f"./Karate/solver-louv_runs-{num_runs}"
louv_communities, louv_modularities, louv_times_elapsed = louv_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

100%|██████████| 50/50 [00:00<00:00, 221.11it/s]


In [30]:
dqm_sampler = DQMSampler(K, resolution=resolution, time=5, cases=4)
dqm_iterative= IterativeSearcher(dqm_sampler)

saving_path = f"./Karate/solver-dqm_runs-{num_runs}"
dqm_communities, dqm_modularities, dqm_times_elapsed = dqm_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

100%|███████████████████████████████████████████| 50/50 [10:40<00:00, 12.81s/it]


In [5]:
leid_sampler = LeidenSampler(K)
leid_iterative= IterativeSearcher(leid_sampler)

saving_path = f"./Karate/solver-leid_runs-{num_runs}"
leid_communities, leid_modularities, leid_times_elapsed = leid_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

100%|██████████| 50/50 [00:00<00:00, 363.78it/s]


In [ ]:
# TODO: Add Bayan, but I cannot install bayanpy
baya_sampler = BayanSampler(K)
baya_iterative= IterativeSearcher(baya_sampler)

saving_path = f"./Karate/solver-baya_runs-{num_runs}"
baya_communities, baya_modularities, baya_times_elapsed = baya_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

In [34]:
# We load the data for reproducibility
adv_modularities = np.load(f"./Karate/solver-adv_runs-{num_runs}.npy")
dqm_modularities = np.load(f"./Karate/solver-dqm_runs-{num_runs}.npy")
louv_modularities = np.load(f"./Karate/solver-louv_runs-{num_runs}.npy")
leid_modularities = np.load(f"./Karate/solver-leid_runs-{num_runs}.npy")

adv_times_elapsed = np.load(f"./Karate/solver-adv_runs-{num_runs}_times.npy")
dqm_times_elapsed = np.load(f"./Karate/solver-dqm_runs-{num_runs}_times.npy")
louv_times_elapsed = np.load(f"./Karate/solver-louv_runs-{num_runs}_times.npy")
leid_times_elapsed = np.load(f"./Karate/solver-leid_runs-{num_runs}_times.npy")

adv_comms = np.load(f"./Karate/solver-adv_runs-{num_runs}_comms.npy", allow_pickle=True)
dqm_comms = np.load(f"./Karate/solver-dqm_runs-{num_runs}_comms.npy", allow_pickle=True)
louv_comms = np.load(f"./Karate/solver-louv_runs-{num_runs}_comms.npy", allow_pickle=True)
leid_comms = np.load(f"./Karate/solver-leid_runs-{num_runs}_comms.npy", allow_pickle=True)

mod_adv, freq_adv = np.unique(adv_modularities, return_counts=True)
mod_dqm, freq_dqm = np.unique(dqm_modularities, return_counts=True)
mod_louv, freq_louv = np.unique(louv_modularities, return_counts=True)
mod_leid, freq_leid = np.unique(leid_modularities, return_counts=True)
#mod_baya, freq_baya = np.unique(baya_modularities, return_counts=True)

c_adv = len(adv_comms[np.argmax(adv_modularities)])
c_dqm = len(dqm_comms[np.argmax(dqm_modularities)])
c_louv = len(louv_comms[np.argmax(louv_modularities)])
c_leid = len(leid_comms[np.argmax(leid_modularities)])

In [35]:
print("Algorithm \t Max Q \t N communities \t Frequency \t Time (s)")
print("---------------------------------------------------------")
print(f"Annealing \t {round(mod_adv[-1],6)} \t {c_adv} \t {100*freq_adv[-1]/num_runs} %  \t {adv_times_elapsed.mean()} +- {adv_times_elapsed.std()/np.sqrt(num_runs)}")
print(f"DQM       \t {round(mod_dqm[-1],6)} \t {c_dqm} \t {100*freq_dqm[-1]/num_runs} %  \t {dqm_times_elapsed.mean()} +- {dqm_times_elapsed.std()/np.sqrt(num_runs)}")
print(f"Louvain   \t {round(mod_louv[-1],6)} \t {c_louv} \t {100*freq_louv[-1]/num_runs} % \t {louv_times_elapsed.mean()} +- {louv_times_elapsed.std()/np.sqrt(num_runs)}")
print(f"Leiden    \t {round(mod_leid[-1],6)} \t {c_leid} \t {100*freq_leid[-1]/num_runs} % \t {leid_times_elapsed.mean()} +- {leid_times_elapsed.std()/np.sqrt(num_runs)}")
#print(f"Bayan    \t {round(mod_baya[-1],6)} \t {100*freq_baya[-1]/num_runs} % \t")

Algorithm 	 Max Q 	 N communities 	 Frequency 	 Time (s)
---------------------------------------------------------
Annealing 	 0.444904 	 4 	 100.0 %  	 8.46568461894989 +- 0.2571043145026076
DQM       	 0.444904 	 4 	 100.0 %  	 12.808557538986205 +- 0.03540523012881962
Louvain   	 0.444904 	 4 	 40.0 % 	 0.0037561511993408204 +- 0.0004049010300129798
Leiden    	 0.444904 	 4 	 100.0 % 	 0.0019035482406616212 +- 5.835346869877643e-05
